In [6]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import types
import torch
import utils
import TextNet
import TeacherNet
import ImageNet
import torch.optim as optim
import time
import sys
import numpy as np
from torch.autograd import Variable


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [7]:
train_img = torch.load("../cached_data/train_img")
train_cap = torch.load("../cached_data/train_cap")
train_mask = torch.load("../cached_data/train_mask")

val_img = torch.load("../cached_data/val_img")
val_cap = torch.load("../cached_data/val_cap")
val_mask = torch.load("../cached_data/val_mask")

print("Loaded train data", train_img.size(), train_cap.size(), train_mask.size())
print("Loaded val data", val_img.size(), val_cap.size(), val_mask.size())



Loaded train data torch.Size([10000, 3, 224, 224]) torch.Size([10000, 52]) torch.Size([10000, 52])
Loaded val data torch.Size([5000, 3, 224, 224]) torch.Size([5000, 43]) torch.Size([5000, 43])


In [8]:
DELTA = 0.1
BATCH_SIZE = 8
NB_EPOCHS = 10
LOGGER = utils.Logger()

# VOCAB_SIZE = len(train_mask)
# EMBED_DIM = train_cap.size()[1]
# NUN_CLASS = len(train_cap)
# model = TextNet.Text_Net(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)


In [9]:
train_data = TensorDataset(train_img, train_cap, train_mask)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, num_workers=2)

valid_data = TensorDataset(val_img, val_cap, val_mask)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE * 2, num_workers=2)


device = torch.device("cuda:1")
text_net = TextNet.Text_Net()
#text_net.to(device)
#text_net = TextNet.Text_Net(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)


model_name = "alexnet"
feature_extract = True
vision_net = ImageNet.Image_Net()
vision_net = vision_net.initialize_model(model_name, feature_extract, use_pretrained=True)
vision_net.to(device)


teacher_net = TeacherNet.Teacher_Net()
ranking_loss = TeacherNet.RankingLossFunc(DELTA)
teacher_net.to(device)
ranking_loss.to(device)

print(text_net)
print(vision_net)

embeds = Variable(torch.Tensor((BATCH_SIZE, 16, 300)), requires_grad=True)

4096
Text_Net(
  (conv1): Conv1d(52, 300, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(300, 300, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(300, 300, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=21900, out_features=9216, bias=True)
)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxP

In [11]:
# optimizer
params_to_update_share = []
params_to_update_img = vision_net.parameters()
params_to_update_txt = []


params_to_update = list(params_to_update_share) + list(params_to_update_img) + list(params_to_update_txt)
optimizer = optim.Adam(params_to_update, lr=0.0001)


print("Start to train")
start_time = time.time()
for epoch in range(NB_EPOCHS):
    running_loss = 0.0
    running_corrects = 0.0
    total_samples = 0

    for step, batch in enumerate(train_dataloader):
        img, cap, mask = tuple(t.to(device, dtype=torch.float) for t in batch)
        print(img.size())
        print(cap.size())
        
        with torch.set_grad_enabled(False):
            img_feature = vision_net.forward(img)
            print(img_feature.size()) 
#             cap = np.expand_dims(cap.cpu(), axis=2)
#             cap=torch.tensor(cap)
#             print(cap.size(), cap.type)
            txt_feature = text_net.forward(cap)
            
        with torch.set_grad_enabled(True):
            img_vec = teacher_net.forward(img_feature)
            txt_vec = teacher_net.forward(txt_feature)
    
            loss = ranking_loss(img_vec, txt_vec)
            preds = teacher_net.predict(img_vec, txt_vec)
            optimizer.step()
            optimizer.zero_grad()
    
        running_loss += loss.item() * BATCH_SIZE
        running_corrects += sum([(i == preds[i]) for i in range(len(preds))])
        total_samples += len(preds)
        #print(running_loss)
        
    print("Epoch %d: train loss = %f" % (epoch, running_loss))
    print(("          train acc = %f (%d/%d)" % (float(running_corrects/total_samples), running_corrects, total_samples)))

print("Training done in %f mins" % ((time.time()-start_time)/60))

Start to train
torch.Size([8, 3, 224, 224])
torch.Size([8, 52])
torch.Size([8, 9216])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight 300 52 3, but got 2-dimensional input of size [8, 52] instead

In [ ]:
print(preds)

In [ ]:
txt_feature = text_net.forward(cap,mask)

In [ ]:
print (txt_feature.size())


In [ ]:
print(cap.size())

In [ ]:
import TextNet 
text_net2 = TextNet.Text_Net()
txt_feature2 = text_net2.forward(cap)